<a href="https://colab.research.google.com/github/MarcoUMartinez/CuTonala_2024_A/blob/MarcoMartinez/Algoritmo_InsertSort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Implementación del algoritmo Insert Sort con paralelismo en MPI4PY y NUMBA**

## **Codigo sin paralelizar**

In [21]:
#########    Algoritmo de Insert Sort   ###############
##
##
##              Autor: Martinez Pérez Marco Uriel

import time

"""
Docstring: Algoritmo InsertSort

Returns:
        list: Lista de valores ordenados

"""

inicio = time.time()  # Guarda el tiempo de inicio

def insertion_sort(arr):
    n = len(arr)
    for i in range(1, n):
        current_value = arr[i]  # Guarda el valor actual a insertar
        position = i

        # Mueve los elementos del subarreglo ordenado hacia la derecha
        # para hacer espacio para el valor actual
        while position > 0 and arr[position - 1] > current_value:
            arr[position] = arr[position - 1]
            position -= 1

        # Inserta el valor actual en la posición correcta
        arr[position] = current_value


# Ejemplo de Arreglo
arr = [5, 2, 4, 6, 1, 3, 8, 10, 32, 7, 1, 20, 9, 12]
print("Lista original:", arr)

insertion_sort(arr)
print("Lista ordenada:", arr)

fin = time.time()  # Guarda el tiempo de finalización
tiempo_ejecucion = fin - inicio  # Calcula el tiempo de ejecución
print(f"Tiempo de ejecución: {tiempo_ejecucion:.8f} segundos")


Lista original: [5, 2, 4, 6, 1, 3, 8, 10, 32, 7, 1, 20, 9, 12]
Lista ordenada: [1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 20, 32]
Tiempo de ejecución: 0.00437117 segundos


## **Codigo MPI4PY**

In [10]:
!pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 13.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.6-cp310-cp310-linux_x86_64.whl size=2746291 sha256=a215df32c53407e605f92fc10efed6f0bd6f2f6d3e4cf7c68b0a4e1996cea680
  Stored in directory: /root/.cache/pip/wheels/4c/ca/89/8fc1fb1c620afca13bb41c630b1f948bbf446e0aaa4b762e10
Successfully built mpi4py


In [19]:
#########    Algoritmo de Insert Sort implementando Mpi4py   ###############
##
##
##              Autor: Martinez Pérez Marco Uriel

from mpi4py import MPI

# Registrar el tiempo de inicio
inicio = MPI.Wtime()
def insertion_sort(arr):
    n = len(arr)
    for i in range(1, n):
        current_value = arr[i]  # Guarda el valor actual a insertar
        position = i

        # Mueve los elementos del subarreglo ordenado hacia la derecha
        # para hacer espacio para el valor actual
        while position > 0 and arr[position - 1] > current_value:
            arr[position] = arr[position - 1]
            position -= 1

        # Inserta el valor actual en la posición correcta
        arr[position] = current_value

# Función para dividir el arreglo en partes iguales
def split_array(arr, size):
    n = len(arr)
    part_size = n // size
    remainder = n % size
    send_counts = [part_size + 1 if i < remainder else part_size for i in range(size)]
    displacements = [sum(send_counts[:i]) for i in range(size)]
    return [arr[displacements[i]:displacements[i] + send_counts[i]] for i in range(size)]

# Función para unir las partes ordenadas
def merge_sorted_parts(sorted_parts):
    return sorted(sum(sorted_parts, []))

if __name__ == "__main__":
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    if rank == 0:
        arr = [5, 2, 4, 6, 1, 3, 8, 10, 32, 7, 1, 20, 9, 12]
    else:
        arr = None

    # Dividir el arreglo en partes iguales
    arr_part = comm.scatter(split_array(arr, size), root=0)

    # Ordenar la parte local
    insertion_sort(arr_part)

    # Recopilar partes ordenadas
    sorted_parts = comm.gather(arr_part, root=0)

    # Unir partes ordenadas
    if rank == 0:
        sorted_arr = merge_sorted_parts(sorted_parts)
        print("Lista original:", arr)
        print("Lista ordenada:", sorted_arr)
        fin = MPI.Wtime()  # Registrar el tiempo de finalización
        print(f"Tiempo de ejecución: {fin - inicio:.8f} segundos")

Lista original: [5, 2, 4, 6, 1, 3, 8, 10, 32, 7, 1, 20, 9, 12]
Lista ordenada: [1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 20, 32]
Tiempo de ejecución: 0.00456926 segundos


## **Codigo NUMBA**

In [1]:
!pip install numba

In [18]:
#########    Algoritmo de Insert Sort implementando NUMBA   ###############
##
##
##              Autor: Martinez Pérez Marco Uriel

from numba import njit
import numpy as np
import time

inicio = time.time()  # Registrar el tiempo de inicio
@njit
def insertion_sort(arr):
    n = len(arr)
    for i in range(1, n):
        current_value = arr[i]
        position = i

        while position > 0 and arr[position - 1] > current_value:
            arr[position] = arr[position - 1]
            position -= 1

        arr[position] = current_value

# Función para dividir el arreglo en partes iguales
def split_array(arr, size):
    n = len(arr)
    part_size = n // size
    remainder = n % size
    send_counts = [part_size + 1 if i < remainder else part_size for i in range(size)]
    displacements = [sum(send_counts[:i]) for i in range(size)]
    return [arr[displacements[i]:displacements[i] + send_counts[i]] for i in range(size)]

# Función para fusionar dos partes ordenadas
@njit
def merge_two_sorted_arrays(arr1, arr2):
    sorted_arr = np.empty(len(arr1) + len(arr2), dtype=arr1.dtype)
    i = j = k = 0

    while i < len(arr1) and j < len(arr2):
        if arr1[i] < arr2[j]:
            sorted_arr[k] = arr1[i]
            i += 1
        else:
            sorted_arr[k] = arr2[j]
            j += 1
        k += 1

    while i < len(arr1):
        sorted_arr[k] = arr1[i]
        i += 1
        k += 1

    while j < len(arr2):
        sorted_arr[k] = arr2[j]
        j += 1
        k += 1

    return sorted_arr

# Función para unir las partes ordenadas
def merge_sorted_parts(sorted_parts):
    while len(sorted_parts) > 1:
        new_sorted_parts = []
        for i in range(0, len(sorted_parts), 2):
            if i + 1 < len(sorted_parts):
                merged_part = merge_two_sorted_arrays(sorted_parts[i], sorted_parts[i + 1])
                new_sorted_parts.append(merged_part)
            else:
                new_sorted_parts.append(sorted_parts[i])
        sorted_parts = new_sorted_parts
    return sorted_parts[0]

if __name__ == "__main__":
    arr = np.array([5, 2, 4, 6, 1, 3, 8, 10, 32, 7, 1, 20, 9, 12], dtype=np.int32)
    size = 4  # Número de partes en las que se dividirá el arreglo

    # Dividir el arreglo en partes iguales
    parts = split_array(arr, size)

    # Ordenar cada parte localmente usando Numba
    for part in parts:
        insertion_sort(part)

    # Unir partes ordenadas
    sorted_arr = merge_sorted_parts(parts)

    fin = time.time()  # Registrar el tiempo de finalización

    print("Lista original:", arr)
    print("Lista ordenada:", sorted_arr)
    print(f"Tiempo de ejecución: {fin - inicio:.8f} segundos")

Lista original: [ 2  4  5  6  1  3  8 10  1  7 32  9 12 20]
Lista ordenada: [ 1  1  2  3  4  5  6  7  8  9 10 12 20 32]
Tiempo de ejecución: 0.453080 segundos
